In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.5
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import pdo_functions
import importlib
importlib.reload(pdo_functions)
import random
import numba
import statsmodels.api as sm
import glob
import pandas as pd
from sklearn.metrics import r2_score
from wrf import interplevel

In [ ]:
# u = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/u/*s/e5.oper.an.pl.128_131_u.*.nc', combine='nested', concat_dim='time')['U']
v = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/v/*s/e5.oper.an.pl.128_132_v.*.nc', combine='nested', concat_dim='time')['V']
# z_1940s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/1940s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']
# z_1950s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/1950s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']
# z_1960s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/1960s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']
# z_1970s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/1970s/1970s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']
# z_1980s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/1980s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']
# z_1990s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/1990s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']
# z_2000s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/2000s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']
# z_2010s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/2010s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']
# z_2020s = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/Z/2020s/e5.oper.an.pl.128_129_z.*.nc', combine='nested', concat_dim='time')['Z']

In [ ]:
# Concatenate time periods
# u = xr.concat([u_1940s, u_1950s, u_1960s, u_1970s, u_1980s, u_1990s, u_2000s, u_2010s, u_2020s], dim='time')

In [ ]:
# Only need MAM
# u_mam = u.sel(time=u.time.dt.month.isin([3, 4, 5]))
v_mam = v.sel(time=v.time.dt.month.isin([3, 4, 5]))

In [ ]:
# Select the correct CONUS grid points
# u_mam_conus = u_mam.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))
v_mam_conus = v_mam.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))

In [ ]:
# Select only 850 mb level
# u_mam_conus_850 = u_mam_conus.sel(level=500)
v_mam_conus_850 = v_mam_conus.sel(level=500)

In [ ]:
v_mam_conus_monthly = np.zeros((255, 101, 237))
# v_mam_conus_monthly = np.zeros((255, 101, 237))
for i in range(85):
    # u_mam_conus_monthly[i*3: (i+1)*3] = u_mam_conus_850.isel(time=slice(i*92, (i+1)*92)).resample(time='ME').mean().dropna(dim='time')
    v_mam_conus_monthly[i*3: (i+1)*3] = v_mam_conus_850.isel(time=slice(i*92, (i+1)*92)).resample(time='ME').mean().dropna(dim='time')
    print(f'{1940+i} is complete.')

In [ ]:
dates = pd.date_range('1940-01-01', '2025-01-01', freq='MS')
dates_mam = dates[dates.month.isin([3, 4, 5])]

In [ ]:
# Convert monthly data to XR
# u_mam_conus_monthly_xr = xr.DataArray(u_mam_conus_monthly, coords={'time':dates_mam, 'latitude':u_mam_conus['latitude'], 'longitude':u_mam_conus['longitude']}, \
#                                       dims=['time', 'latitude', 'longitude'])

v_mam_conus_monthly_xr = xr.DataArray(v_mam_conus_monthly, coords={'time':dates_mam, 'latitude':u_mam_conus['latitude'], 'longitude':u_mam_conus['longitude']}, \
                                      dims=['time', 'latitude', 'longitude'])

In [ ]:
# Save as a netcdf
# u_mam_conus_monthly_xr.to_netcdf('/hurrell-scratch2/ivyglade/pdo/ERA5_large_scale_envt/u_500_monthly/u_500_mam_1940-2024.nc')
v_mam_conus_monthly_xr.to_netcdf('/hurrell-scratch2/ivyglade/pdo/ERA5_large_scale_envt/v_500_monthly/v_500_mam_1940-2024.nc')